In [75]:
import pandas as pd
import numpy as np
from keras.utils import np_utils

In [76]:
df = pd.read_csv("iris.csv")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [77]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [78]:
features = list(df.columns[:4])

x = df[features]
print(x.head())

x = np.array(x)


y = list(df['species'])

y = np.array(y)

   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2


### Label encoding

The labels vector contains labels for the flowers in a single dimension. Using the LabelEncoder we convert that to 3 dimensions each  with 1 or 0 according to whether or not they fall into any of the 3 types

In [79]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y1 = encoder.fit_transform(y)
y = pd.get_dummies(y1).values

# Alternative for checking the head in ndarray
print(y[:5])

[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]]


### Train/Test Split

We us the train_test_split function in the model_selction module to split the data as follows:

 - 30% test data
 - 70% training data

In [80]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.30, random_state=0)

### Neural Network Architecture

Below we are going to define using keras a neural network model with the following layers

 - Layer 1: Input layer with 4 units (feature matrix has 4 dimensions) RELU activation
 - Layer 2: Inner layer with 5 units RELU activation
 - Layer 3: Output layer with 3 units SOFTMAX activation

This is Sequential model is created incrementally via the add() method. 

In [81]:
import keras

# Define model with keras_Squential
model = keras.Sequential()

# Using 4 units at input as the feature matrix has 4 dimensions
model.add(Dense(units = 4, activation = 'relu'))

# Inner layer defined with 5 units
model.add(Dense(units = 5, activation = 'relu'))

# Using 3 output units as the label matrix has 3 dimensions
model.add(Dense(units = 3, activation = 'softmax'))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=200, batch_size=5, shuffle=True)


curacy: 0.6421
Epoch 6/200
21/21 [==============================] - 0s 650us/step - loss: 0.9270 - accuracy: 0.6332
Epoch 7/200
21/21 [==============================] - 0s 700us/step - loss: 0.9087 - accuracy: 0.6279
Epoch 8/200
21/21 [==============================] - 0s 650us/step - loss: 0.7817 - accuracy: 0.7405
Epoch 9/200
21/21 [==============================] - 0s 699us/step - loss: 0.7759 - accuracy: 0.7188
Epoch 10/200
21/21 [==============================] - 0s 600us/step - loss: 0.7329 - accuracy: 0.6773
Epoch 11/200
21/21 [==============================] - 0s 750us/step - loss: 0.7435 - accuracy: 0.7993
Epoch 12/200
21/21 [==============================] - 0s 650us/step - loss: 0.7063 - accuracy: 0.7390
Epoch 13/200
21/21 [==============================] - 0s 700us/step - loss: 0.6901 - accuracy: 0.7571
Epoch 14/200
21/21 [==============================] - 0s 650us/step - loss: 0.6965 - accuracy: 0.6990
Epoch 15/200
21/21 [==============================] - 0s 652us/step - l

In [82]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (5, 4)                    20        
_________________________________________________________________
dense_37 (Dense)             (5, 5)                    25        
_________________________________________________________________
dense_38 (Dense)             (5, 3)                    18        
Total params: 63
Trainable params: 63
Non-trainable params: 0
_________________________________________________________________


In [83]:
scores = model.evaluate(x_test, y_test)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

2/2 [==============================] - 0s 2ms/step - loss: 0.1416 - accuracy: 0.9556

Accuracy: 95.56%


In [88]:
y_pred = model.predict(x_test)

In [86]:
x_test[1,:],y_pred[1]

(array([6. , 2.2, 4. , 1. ]),
 array([6.5911474e-04, 9.9854064e-01, 8.0022356e-04], dtype=float32))

In [36]:
y_pred_class = np.argmax(y_pred[1])
print(y_pred_class)

1


In [37]:
model.save('model_file.iris')

INFO:tensorflow:Assets written to: model_file.iris\assets


In [ ]:
import numpy as np 
from keras.models import load_model

In [55]:
iris_model = load_model('model_file.iris')



In [93]:
T = np.array([4.9, 3.0, 1.4, 0.9])
#data = T.transpose()
print(data.shape)


(4,)


In [92]:
predictions = iris_model.predict(data)

ValueError: in user code:

    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1461 run_step  **
        outputs = model.predict_step(data)
    c:\python38\lib\site-packag

In [71]:
predictions = iris_model.predict([6,2,1.8,2])

ValueError: in user code:

    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1461 run_step  **
        outputs = model.predict_step(data)
    c:\python38\lib\site-packag